In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#para cargar y visualizar los datos de audio
import os, glob
import librosa
import librosa.display
from librosa.display import specshow

#para crear las imagenes de los espectrogramas
import matplotlib.pyplot as plt

#para cargar las imagenes
import cv2

import time
import numpy as np
import pandas as pd

import keras
from keras.applications import vgg16

from sklearn.cluster import KMeans

In [ ]:
#funcion para convertir los audios a imagenes de espactrogramas
#y guarda las imagenes en la misma carpeta de los audios
def convert_to_spec_image(file):
  x, sample_rate = librosa.load(file)

  plt.figure(figsize=(10,3))
  
  stft = librosa.stft(x)
  stft = librosa.amplitude_to_db(abs(stft))
  specshow(stft, sr=sample_rate, x_axis='time', y_axis='hz')  
  
  plt.ylim(0, 5000)

  file_img = file.replace(".wav",".png")

  plt.savefig(file_img)

  plt.close()
  

In [ ]:
#funcion para cargar los audio
def load_audio():
  x = []
  labels_x = []

  for file in glob.glob("/content/drive/Shareddrives/Reconocimiento de Voz Drive/AudioData/*/*.wav"):
    file_name = os.path.basename(file)
    feature = convert_to_spec_image(file)

In [ ]:
#cargar y convierte los audios a imagenes
load_audio()

In [ ]:
#funcion para cargar las imagenes y las etiquetas
def load_images():
    
    # Definimos matrices vacías donde almacenaremos nuestras imágenes y etiquetas
    images = []
    labels = []
    
    for file in glob.glob("/content/drive/Shareddrives/Reconocimiento de Voz Drive/AudioData/*/*.png"):

            file_name = os.path.basename(file)

            # Leemos la imagen
            image = cv2.imread(file)
            
            # Cambiamos el tamaño a 224 x 224
            image = cv2.resize(image, (224,224))
            
            # Convertimos de BGR a RGB para que podamos trazarlos más tarde (porque openCV lee las imágenes como BGR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Ahora lo agregamos a nuestra matriz
            images.append(image)
            labels.append(file_name[:-6])

    return images, labels

In [ ]:
#carga las imagenes y las etiquetas
images, labels = load_images()
#crea un lista de etiqutas unicas
codes = set(labels)

In [ ]:
print(codes)
print(len(codes))

{'h', 'a', 'd', 'n', 'sa', 'f', 'su'}
7


In [ ]:
def normalise_images(images, labels):
   
    # Convertimos en arreglos para Numpy
    images = np.array(images, dtype=np.float32) 
    labels = np.array(labels)
    
    # Normalizamos las imágenes
    images /= 255
    
    return images, labels

In [ ]:
images, labels = normalise_images(images, labels)

In [ ]:
labels

array(['n', 'su', 'n', 'sa', 'n', 'sa', 'sa', 'n', 'sa', 'su', 'su', 'n',
       'n', 'sa', 'su', 'n', 'n', 'su', 'sa', 'n', 'n', 'n', 'n', 'n',
       'su', 'h', 'su', 'n', 'n', 'su', 'n', 'n', 'n', 'sa', 'n', 'n',
       'su', 'n', 'n', 'n', 'n', 'n', 'h', 'su', 'n', 'sa', 'su', 'sa',
       'a', 'h', 'sa', 'su', 'n', 'a', 'a', 'su', 'sa', 'a', 'n', 'n',
       'h', 'sa', 'd', 'sa', 'd', 'f', 'd', 'sa', 'su', 'a', 'su', 'f',
       'sa', 'h', 'f', 'f', 'f', 'd', 'h', 'h', 'd', 'h', 'f', 'n', 'h',
       'a', 'a', 'd', 'h', 'f', 'd', 'a', 'h', 'f', 'a', 'f', 'h', 'h',
       'f', 'h', 'f', 'd', 'f', 'a', 'a', 'h', 'd', 'd', 'd', 'd', 'd',
       'd', 'f', 'f', 'd', 'a', 'a', 'f', 'a', 'a', 'su', 'sa', 'su',
       'sa', 'su', 'su', 'sa', 'su', 'n', 'su', 'sa', 'sa', 'n', 'n',
       'sa', 'sa', 'sa', 'su', 'n', 'n', 'su', 'n', 'sa', 'n', 'su', 'su',
       'n', 'n', 'su', 'n', 'su', 'sa', 'n', 'sa', 'sa', 'sa', 'n', 'sa',
       'f', 'h', 'sa', 'h', 'n', 'd', 'f', 'su', 'f', 'f', 'd',

In [ ]:
#creamos un modelo de red neuronal vgg16
model = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224,224,3))

In [ ]:
# se usara una red preentrenada como parte del preprocesamiento
def Vgg16(model_vgg16, raw_images):

    # Pasamos nuestros datos de entrenamiento a través de la red   
    pred = model_vgg16.predict(raw_images)
    
    # aplanamos la matriz
    flat = pred.reshape(raw_images.shape[0], -1)
    
    return flat

In [ ]:
#reducimos la simencion de nuestros datos usando vgg16
vgg16 = Vgg16(model,images)
print("VGG16 flattened output has {} features".format(vgg16.shape[1]))


VGG16 flattened output has 25088 features


In [ ]:
#funcion para entrenar el modelo usando clusters
def create_train_kmeans(data, number_of_clusters=len(codes)):
    k = KMeans(n_clusters=number_of_clusters, random_state=728)

    # Haremos algunos horarios para ver cuánto tiempo lleva entrenar.
    start = time.time()

    # Entrenarlo
    k.fit(data)

    # Detener el tiempo
    end = time.time()

    # Y miramos cuánto tiempo nos tomó esto
    print("Training took {} seconds".format(end-start))
    
    return k

In [ ]:
# Aquí creamos y ajustamos un modelo KMeans con la salida de vgg16
print("KMeans vgg16: \n")
K_means = create_train_kmeans(vgg16)

KMeans vgg16: 

Training took 10.948641061782837 seconds


In [ ]:
# Ahora tenemos las predicciones del modelo cluster

# KMeans con salidas de vgg16
k_pred = K_means.predict(vgg16)

In [ ]:
def cluster_label_count(clusters, labels):
    
    count = {}

    # Obtengamos los clústeres y las etiquetas
    unique_clusters = list(set(clusters))
    unique_labels = list(set(labels))
    
    # Creamos un contador para cada combinación del clúster sobre(/) etiqueta y establecerlo en 0
    for cluster in unique_clusters:
        count[cluster] = {}
        
        for label in unique_labels:
            count[cluster][label] = 0
    
    # Contemos
    for i in range(len(clusters)):
        count[clusters[i]][labels[i]] +=1
    
    cluster_df = pd.DataFrame(count)
    
    return cluster_df

In [ ]:
# contamos los clústeres para la prediccion VGG16
cluster_count = cluster_label_count(k_pred, labels)

In [ ]:
print("KMeans VGG16: ")
cluster_count

KMeans VGG16: 


,0,1,2,3,4,5,6
h,2,19,6,21,7,0,5
a,1,24,8,15,5,1,6
d,2,21,6,6,7,12,6
n,2,23,9,6,20,54,6
sa,3,21,10,10,4,7,5
f,3,18,7,21,5,0,6
su,1,13,6,28,8,0,4


In [ ]:
# Ajustamos manualmente estas listas para que el índice de cada etiqueta refleje en qué cluster se encuentra
cluster_code = ["f","a","sa","su","h","n","d"]

In [ ]:
pred_codes = [cluster_code[x] for x in k_pred]
pred_codes

['h',
 'h',
 'h',
 'sa',
 'sa',
 'sa',
 'sa',
 'h',
 'sa',
 'h',
 'h',
 'sa',
 'h',
 'h',
 'f',
 'h',
 'h',
 'sa',
 'h',
 'h',
 'f',
 'h',
 'f',
 'h',
 'h',
 'sa',
 'sa',
 'h',
 'sa',
 'sa',
 'sa',
 'h',
 'h',
 'sa',
 'h',
 'h',
 'h',
 'h',
 'h',
 'sa',
 'sa',
 'sa',
 'sa',
 'h',
 'sa',
 'sa',
 'sa',
 'h',
 'sa',
 'f',
 'f',
 'h',
 'h',
 'sa',
 'sa',
 'sa',
 'f',
 'sa',
 'h',
 'h',
 'h',
 'sa',
 'sa',
 'sa',
 'f',
 'f',
 'h',
 'sa',
 'sa',
 'h',
 'h',
 'f',
 'sa',
 'h',
 'sa',
 'h',
 'h',
 'h',
 'sa',
 'sa',
 'f',
 'sa',
 'sa',
 'sa',
 'h',
 'd',
 'sa',
 'sa',
 'h',
 'h',
 'sa',
 'sa',
 'h',
 'sa',
 'sa',
 'h',
 'f',
 'h',
 'sa',
 'sa',
 'f',
 'sa',
 'sa',
 'h',
 'h',
 'h',
 'sa',
 'h',
 'h',
 'h',
 'sa',
 'h',
 'h',
 'sa',
 'h',
 'f',
 'sa',
 'sa',
 'h',
 'h',
 'su',
 'n',
 'su',
 'n',
 'su',
 'a',
 'a',
 'su',
 'n',
 'su',
 'a',
 'n',
 'h',
 'd',
 'a',
 'a',
 'su',
 'su',
 'n',
 'n',
 'su',
 'n',
 'd',
 'n',
 'a',
 'a',
 'n',
 'a',
 'su',
 'n',
 'd',
 'f',
 'n',
 'h',
 'n',
 'n',
 'n

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def print_scores(true, pred):
    acc = accuracy_score(true, pred)
    f1 = f1_score(true, pred, average="macro")
    ps = precision_score(true, pred, average="macro")
    rs = recall_score(true, pred, average="macro")

    return "\n\tF1 Score: {0:0.8f} | Accuracy: {0:0.8f} | Precision: {0:0.8f} | Recall: {0:0.8f}".format(f1,acc,ps,rs)

print("KMeans VGG16:", print_scores(labels, pred_codes))

KMeans VGG16: 
	F1 Score: 0.23371822 | Accuracy: 0.23371822 | Precision: 0.23371822 | Recall: 0.23371822
